In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'font.size': 12})

from scipy import signal 
from scipy import stats
import itertools
import seaborn as sns
import statsmodels.api as sm

import os
import random

In [7]:
import pickle
import json
json_open = open('../../dj_local_conf.json', 'r') 
config = json.load(json_open)

import datajoint as dj
dj.config['database.host'] = config["database.host"]
dj.config['database.user'] = config ["database.user"]
dj.config['database.password'] = config["database.password"]
dj.conn().connect()

from pipeline import lab, get_schema_name, experiment, foraging_model, ephys, foraging_analysis, histology, ccf
from pipeline.plot import unit_psth
from pipeline.plot.foraging_model_plot import plot_session_model_comparison, plot_session_fitted_choice
from pipeline import psth_foraging
from pipeline import util
from pipeline.model import bandit_model

from UtilFunctions_KH import _get_independent_variableKH, align_phys_to_behav_trials

In [13]:
DATA_ROOT_DIR = '../../data/nm_data'
ANA_DIR = 'FB26_220715/'
ana_dir_path = os.path.join(DATA_ROOT_DIR, ANA_DIR)

with open(os.path.join(ana_dir_path, 'data_frame.pkl'), 'rb') as fh:
    df = pickle.load(fh)

sess_key = {'subject_id': 627713, 'session': 23}

### Standard analysis